# Signpost CNN Classification

In this kernel I’ll try to identify correctly which signpost we are seeing ( 43 types of signposts).
In order to do so I’ll use signpost image dataset as input to a CNN (since we have huge data set - no need for data augmentation  ) .
After optimzation process - optimal layers are 2 + 2 FC layers.
I also run semi grid search on dropout , filter size etc..

# Import

In [ ]:
import numpy as np 
import pandas as pd 
import pickle
import matplotlib.pyplot as plt
from math import sqrt, ceil
from timeit import default_timer as timer
import os

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler




#  Show input files 

In [ ]:
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Make sure we have enough samples per signpost(43 types) 

In [ ]:
with open('../input/traffic-signs-preprocessed/data1.pickle', 'rb') as f:
    data = pickle.load(f, encoding='latin1')  # dictionary type
pd_check=pd.DataFrame({'y_train': data['y_train']})
pd_check2=pd_check.sort_values(by=['y_train'],ascending=True)
pd_check=pd_check2.set_index(np.arange(len(pd_check2.index)))
#pd_check.reset_index(drop=True, col_level=0)
pd_check['y_train'].value_counts().plot(kind='bar',figsize=(15, 15),color=list('rgbkymc')) 

# Load one of the data set :
* Open file
* Make y(train & val) as categorical list
* Reshape X (test,val & test) to assure RGB image channels (3) comes last

In [ ]:

data['y_train'] = to_categorical(data['y_train'], num_classes=43)
data['y_validation'] = to_categorical(data['y_validation'], num_classes=43)

print("x_train_pre_shape" + ":" ,data['x_train'].shape )
data['x_train'] = data['x_train'].transpose(0, 2, 3, 1)
data['x_validation'] = data['x_validation'].transpose(0, 2, 3, 1)
data['x_test'] = data['x_test'].transpose(0, 2, 3, 1)






# Show dataset size  

In [ ]:

for i, j in data.items():
    if i == 'labels':
        print(i + ':', len(j))
    else: 
        print(i + ':', j.shape)

       

# Showing some examples

In [ ]:

import random
matrix_hw=5
start_idx=random.randint(0,len(data['x_train'])-matrix_hw)

examples = data['x_train'][start_idx:start_idx+matrix_hw**2, :, :, :]
#print(examples.shape)  # (81, 32, 32, 3)
_, axs = plt.subplots(matrix_hw, matrix_hw) #n_row, n_col, figsize=(32, 32,3)
axs = axs.flatten()
for img, ax in zip(examples, axs):
    #print(mg)
    low, high = np.min(img), np.max(img)
    img = 255.0 * (img - low) / (high - low)
    ax.imshow(img.astype('uint8'), cmap='gray')
plt.axis('off')
plt.gcf().set_size_inches(10, 10)
#plt.title('Some examples of training data', fontsize=18)
plt.show()

# Define CNN parameters

In [ ]:
IMAGE_WIDTH    =32
IMAGE_HEIGHT   =32
IMAGE_SIZE     =(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS =3
DROP_OUT_VALUE =[0,0.1,0.2,0,0.1]
POOL_SIZE      =[2,2,2,3,3]
epochs         = 5
batch_size      =5
filters        = [3,3,3,3,3]

# Define CNN model

In [ ]:

model = [0] * len(filters)

for i in range(len(model)):
    model[i] = Sequential()
    model[i].add(Conv2D(32, kernel_size=filters[i], activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
    model[i].add(BatchNormalization())
    model[i].add(MaxPool2D(pool_size=(POOL_SIZE[i],POOL_SIZE[i])))
    model[i].add(Dropout(DROP_OUT_VALUE[i]))
    
    model[i].add(Conv2D(64, kernel_size=filters[i], activation='relu'))
    model[i].add(BatchNormalization())
    model[i].add(MaxPool2D(pool_size=(POOL_SIZE[i],POOL_SIZE[i])))
    model[i].add(Dropout(DROP_OUT_VALUE[i]))
    
    model[i].add(Flatten())
    model[i].add(Dense(512, activation='relu'))
    model[i].add(Dense(43, activation='softmax'))
    model[i].compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model[i].summary()






# Early Stop

To prevent over fitting we will stop the learning after 5 epochs and val_loss value not decreased (1% defualt)

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience=5)


# Learning Rate Reduction

We will reduce the learning rate with some function

In [ ]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** (x + epochs))


# Training Models


In [ ]:

h = [0] * len(model)

for i in range(len(h)):
    h[i] = model[i].fit(data['x_train'], data['y_train'],
                        batch_size=batch_size, epochs = epochs,
                        validation_data = (data['x_validation'], data['y_validation']),
                        callbacks=[annealer,earlystop], verbose=1)
    
    print('Model with filters {0:d}x{0:d},dropout {1:.2f},pooling size {2:d}x{2:d} epochs={3:d}, training acc={4:.5f}, val acc={5:.5f}'.\
      format(filters[i],DROP_OUT_VALUE[i],POOL_SIZE[i], epochs, max(h[i].history['acc']), max(h[i].history['val_acc'])))


# Plotting accuracy per model

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (15.0, 15.0) 
plt.rcParams['image.interpolation'] = 'nearest'

fig = plt.figure()
plt.subplot(2, 1, 1)
plt.plot(h[4].history['acc'], '-o', linewidth=3.0)
plt.plot(h[3].history['acc'], '-o', linewidth=3.0)
plt.plot(h[2].history['acc'], '-o', linewidth=3.0)
plt.plot(h[1].history['acc'], '-o', linewidth=3.0)
plt.plot(h[0].history['acc'], '-o', linewidth=3.0)
plt.legend(['M5', 'M4', 'M3', 'M2', 'M1'], loc='lower right', fontsize='xx-large', borderpad=2)
plt.xlabel('Epoch', fontsize=20, fontname='Times New Roman')
plt.ylabel('Training Accuracy', fontsize=20, fontname='Times New Roman')
plt.yscale('linear')  # {"linear", "log", "symlog", "logit", ...}
plt.ylim(0.85, 1.0)
plt.xlim(0.5, 5.3) 
plt.title('Accuracy for different sizes of filters', fontsize=22)
plt.tick_params(labelsize=18)

plt.subplot(2, 1, 2)
# plt.gca().set_title('Validation accuracy')

plt.plot(h[4].history['val_acc'], '-o', linewidth=3.0)
plt.plot(h[3].history['val_acc'], '-o', linewidth=3.0)
plt.plot(h[2].history['val_acc'], '-o', linewidth=3.0)
plt.plot(h[1].history['val_acc'], '-o', linewidth=3.0)
plt.plot(h[0].history['val_acc'], '-o', linewidth=3.0)
plt.legend(['M5', 'M4', 'M3', 'M2', 'M1'], loc='lower right', fontsize='xx-large', borderpad=2)
plt.xlabel('Epoch', fontsize=20, fontname='Times New Roman')
plt.ylabel('Validation Accuracy', fontsize=20, fontname='Times New Roman')
plt.yscale('linear')  # {"linear", "log", "symlog", "logit", ...}
plt.ylim(0.9, 1)
plt.xlim(0.5, 5.3)
plt.tick_params(labelsize=18)

# Showing the plot
plt.show()

# Saving the plot
fig.savefig('models_acc_cmp.png')
plt.close()




# Predict Xtest 

In [ ]:
for i in range(len(model)):
    temp = model[i].predict(data['x_test'])
    temp = np.argmax(temp, axis=1)
    temp = np.mean(temp == data['y_test'])
    
    print('data2 filter {0:d} testing accuracy = {1:.5f}'.format(filters[i], temp))


# Show example for prediction 

In [ ]:
%matplotlib inline

# Preparing image for predicting from test dataset
x_input = data['x_test'][100:101]
print(x_input.shape)
y_input = data['y_test'][100:101]
print(y_input)

plt.rcParams['figure.figsize'] = (2.5, 2.5) # Setting default size of plots
plt.imshow(x_input[0, :, :, :])
plt.axis('off')

# Showing the plot
plt.show()

# Getting scores from forward pass of input image
scores = model[0].predict(x_input)
print(scores[0].shape) # (43,)

# Scores is given for image with 43 numbers of predictions for each class
# Getting only one class with maximum value
prediction = np.argmax(scores)
print('ClassId:', prediction)

# Defining function for getting texts for every class - labels
def label_text(file):
    # Defining list for saving label in order from 0 to 42
    label_list = []
    
    # Reading 'csv' file and getting image's labels
    r = pd.read_csv(file)
    # Going through all names
    for name in r['SignName']:
        # Adding from every row second column with name of the label
        label_list.append(name)
    
    # Returning resulted list with labels
    return label_list


# Getting labels
labels = label_text('../input/traffic-signs-preprocessed/label_names.csv')

# Printing label for classified Traffic Sign
print('Label:', labels[prediction])


# Save Models

In [ ]:
for i in range(len(model)):
    name = 'model-' + str(filters[i]) + 'x' + str(filters[i]) + '.h5'
    model[i].save(name)


